In [253]:
import os
import sys
import time
import pandas as pd
import asyncio
from binance import Client, AsyncClient, BinanceSocketManager
from binance.exceptions import BinanceAPIException
from pprint import pformat
from binance.helpers import round_step_size
from binance.enums import *

In [254]:
TEST_API_KEY = 'MO83SLsVIwDgypV8XU8k7mZ2EKA1pl6MGHiLkFFAhgd8iZF2OHQM7aj71r8ESNN4'
TEST_API_SECRET = 'YovCpnViZqrai0dHcitwKYSSGOeNya9QPPuzy9zBtFqn3RKOwy7aa0CB3ydSlZdV'


In [255]:
#  Instantiate Binance API client TESTNET US Market
client = Client(api_key=TEST_API_KEY, api_secret=TEST_API_SECRET, testnet=True, tld='us')

In [256]:
def convert_scientific_to_decimal(num):
    if 'e' in str(num):
        return "{:.15f}".format(float(str(num)))
    else:
        return str(num)

In [268]:
#  Get filter info in raw format
info = client.get_symbol_info('BTCUSDT')
print(pformat(info))

{'allowTrailingStop': True,
 'baseAsset': 'BTC',
 'baseAssetPrecision': 8,
 'baseCommissionPrecision': 8,
 'filters': [{'filterType': 'PRICE_FILTER',
              'maxPrice': '1000000.00000000',
              'minPrice': '0.01000000',
              'tickSize': '0.01000000'},
             {'avgPriceMins': 5,
              'filterType': 'PERCENT_PRICE',
              'multiplierDown': '0.2',
              'multiplierUp': '5'},
             {'filterType': 'LOT_SIZE',
              'maxQty': '900.00000000',
              'minQty': '0.00000100',
              'stepSize': '0.00000100'},
             {'applyToMarket': True,
              'avgPriceMins': 5,
              'filterType': 'MIN_NOTIONAL',
              'minNotional': '10.00000000'},
             {'filterType': 'ICEBERG_PARTS', 'limit': 10},
             {'filterType': 'MARKET_LOT_SIZE',
              'maxQty': '100.00000000',
              'minQty': '0.00000000',
              'stepSize': '0.00000000'},
             {'filterType':

In [265]:
symbol_info = {}
def get_symbol_info(symbols):
    global symbol_info
    for symbol in symbols:
        info = client.get_symbol_info(symbol)
        base_asset = info['baseAsset']

        #  PRICE_FILTER
        price_filter_min_quantity = float(info['filters'][0]['minPrice'])
        price_filter_max_quantity = float(info['filters'][0]['maxPrice'])
        price_filter_tick_size = float(info['filters'][0]['tickSize'])
        
        #  LOT_SIZE (for market- and limit orders)
        lot_size_min_quantity = float(info['filters'][2]['minQty'])
        lot_size_max_quantity = float(info['filters'][2]['maxQty'])
        lot_size_step_size = float(info['filters'][2]['stepSize'])
        
        #  MIN NOTIONAL
        min_notional = float(info['filters'][3]['minNotional'])
        

        symbol_info[symbol] = {'price_filter_min_quantity': price_filter_min_quantity, 'price_filter_max_quantity': price_filter_max_quantity,\
                               'price_filter_tick_size': price_filter_tick_size,'lot_size_min_quantity': lot_size_min_quantity, \
                                'lot_size_min_quantity': lot_size_min_quantity, 'lot_size_max_quantity': lot_size_max_quantity, \
                               'lot_size_step_size': lot_size_step_size, 'min_notional': min_notional, }
        
symbols = ['BTCUSDT']
get_symbol_info(symbols)
print(pformat(symbol_info))

{'BTCUSDT': {'lot_size_max_quantity': 900.0,
             'lot_size_min_quantity': 1e-06,
             'lot_size_step_size': 1e-06,
             'min_notional': 10.0,
             'price_filter_max_quantity': 1000000.0,
             'price_filter_min_quantity': 0.01,
             'price_filter_tick_size': 0.01}}


In [258]:
#  Get the balances for both coins
btc_balance = client.get_asset_balance(asset='BTC').get('free')
print('BTC ',btc_balance)
usdt_balance = client.get_asset_balance(asset='USDT').get('free')
print('USDT ',usdt_balance)

BTC  1.00696400
USDT  9495.52236687


In [259]:
#  What's the exchange rate between BTC and USDT? 
price = float(client.get_symbol_ticker(symbol='BTCUSDT')['price'])
print('One BTC can buy ',price,' USDT')

One BTC can buy  40447.71  USDT


In [260]:
# How many BTC do we want to spend? Let's say $0.01 worth
quantity = float(0.01 / price)
print(quantity)

2.4723278524297173e-07


In [261]:
#  Create a Buy Order
order = client.create_order(
    symbol='BTCUSDT',
    quantity=quantity,
    side=SIDE_BUY,
    type=ORDER_TYPE_MARKET,
)

BinanceAPIException: APIError(code=-1100): Illegal characters found in parameter 'quantity'; legal range is '^([0-9]{1,20})(\.[0-9]{1,20})?$'.

In [263]:
#  Convert scientific notation to decimal
quantity = convert_scientific_to_decimal(quantity)
print(quantity)

#  Create a Buy Order
order = client.create_order(
    symbol='BTCUSDT',
    quantity=quantity,
    side=SIDE_BUY,
    type=ORDER_TYPE_MARKET,
)

0.000000247232785


BinanceAPIException: APIError(code=-1111): Precision is over the maximum defined for this asset.

In [271]:
#  Format quantity to a decimal string
quantity = '{:.8f}'.format(float(quantity))
print('quantity: ',quantity)

#  Create a Buy Order
order = client.create_order(
    symbol='BTCUSDT',
    quantity=quantity,
    side=SIDE_BUY,
    type=ORDER_TYPE_MARKET,
)

quantity:  0.00000025


BinanceAPIException: APIError(code=-1013): Filter failure: LOT_SIZE

In [272]:
#  Check quantity filters
info = symbol_info['BTCUSDT']
min_quantity = info['lot_size_min_quantity']
max_quantity = info['lot_size_max_quantity']
print('min_quantity: ',min_quantity,', max_quantity: ',max_quantity,', quantity: ',quantity)
quantity = float(quantity)
if quantity < min_quantity:
    print('Quantity too small')
if quantity > max_quantity:
    print('Quantity too large')


min_quantity:  1e-06 , max_quantity:  900.0 , quantity:  0.00000025
Quantity too small


In [274]:
#  Increase quantity to $0.1
quantity = float(0.1 / price_btc)
quantity = '{:.8f}'.format(float(quantity))
print('quantity: ', quantity)

if float(quantity) < min_quantity:
    print('Quantity too small')


order = client.create_order(
    symbol='BTCUSDT',
    quantity=quantity,
    side=SIDE_BUY,
    type=ORDER_TYPE_MARKET,
)

quantity:  0.00000247


BinanceAPIException: APIError(code=-1013): Filter failure: LOT_SIZE

In [275]:
#  Need to round up quantity to meet step size
step_size = info['lot_size_step_size']
print('step_size: ',step_size)
btc_quantity = round_step_size(quantity, step_size)
quantity = '{:.6f}'.format(float(quantity))
print('quantity', btc_quantity)

order = client.create_order(
    symbol='BTCUSDT',
    quantity=quantity,
    side=SIDE_BUY,
    type=ORDER_TYPE_MARKET,
)

step_size:  1e-06
quantity 2e-06


BinanceAPIException: APIError(code=-1013): Filter failure: MIN_NOTIONAL

In [276]:
# The Min Notional filter defines the minimum value calculated in the quote asset for a symbol.
# For our symbol BTCUSDT the quote symbol is USDT, so the min notional values is in USDT.
# notional_value = price * quantity
min_notional = info['min_notional']
my_notional = price * float(quantity)
print(f"Min Notional {min_notional}, My notional: {my_notional}")
if my_notional < min_notional:
    print('Notional is too small')
    
#  To fix this, we need to again increase our initial investment amount
#  Let's try $10
quantity = float(15 / price)
quantity = round_step_size(quantity, step_size)
quantity = '{:.6f}'.format(float(quantity))

my_notional = price * float(quantity)
print(f"Min Notional {min_notional}, My notional: {my_notional}")
if my_notional < min_notional:
    print('Notional is too small')

Min Notional 10.0, My notional: 0.08089542
Notional is too small
Min Notional 10.0, My notional: 14.9656527


In [277]:
#  Create a Buy Order
order = client.create_order(
    symbol='BTCUSDT',
    quantity=quantity,
    side=SIDE_BUY,
    type=ORDER_TYPE_MARKET,
)
print(pformat(order))

{'clientOrderId': 'mmeixLfR2iPs95zKd8LlZI',
 'cummulativeQuoteQty': '14.93689630',
 'executedQty': '0.00037000',
 'fills': [{'commission': '0.00000000',
            'commissionAsset': 'BTC',
            'price': '40369.99000000',
            'qty': '0.00037000',
            'tradeId': 1388670}],
 'orderId': 4537488,
 'orderListId': -1,
 'origQty': '0.00037000',
 'price': '0.00000000',
 'side': 'BUY',
 'status': 'FILLED',
 'symbol': 'BTCUSDT',
 'timeInForce': 'GTC',
 'transactTime': 1650099083542,
 'type': 'MARKET'}


In [278]:
btc_balance = client.get_asset_balance(asset='BTC').get('free')
print('btc_balance: ',btc_balance)
usdt_balance = client.get_asset_balance(asset='USDT').get('free')
print('usdt_balance: ',usdt_balance)

btc_balance:  1.00733400
usdt_balance:  9480.58547057
